In [6]:
!wget https://resources.lendingclub.com/LoanStats_2019Q1.csv.zip
!wget https://resources.lendingclub.com/LoanStats_2019Q2.csv.zip
!wget https://resources.lendingclub.com/LoanStats_2019Q3.csv.zip
!wget https://resources.lendingclub.com/LoanStats_2019Q4.csv.zip
!wget https://resources.lendingclub.com/LoanStats_2020Q1.csv.zip

--2023-08-31 14:44:48--  https://resources.lendingclub.com/LoanStats_2019Q1.csv.zip
Resolving resources.lendingclub.com (resources.lendingclub.com)... 54.203.181.155, 35.164.180.155, 52.39.166.182
Connecting to resources.lendingclub.com (resources.lendingclub.com)|54.203.181.155|:443... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [application/zip]
Saving to: 'LoanStats_2019Q1.csv.zip'

LoanStats_2019Q1.cs     [               <=>  ]  19.62M  5.73MB/s    in 3.7s    

2023-08-31 14:44:55 (5.24 MB/s) - 'LoanStats_2019Q1.csv.zip' saved [20576260]

--2023-08-31 14:44:55--  https://resources.lendingclub.com/LoanStats_2019Q2.csv.zip
Resolving resources.lendingclub.com (resources.lendingclub.com)... 35.164.180.155, 52.39.166.182, 54.203.181.155
Connecting to resources.lendingclub.com (resources.lendingclub.com)|35.164.180.155|:443... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [application/zip]
Saving to: 'LoanStats_2019Q2.csv.zip'

In [7]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split

In [8]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership", "annual_inc", 
    "verification_status", "pymnt_plan", "dti", "delinq_2yrs", 
    "inq_last_6mths", "open_acc", "pub_rec", "revol_bal", "total_acc", 
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt", 
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", 
    "total_rec_late_fee", "recoveries", "collection_recovery_fee", 
    "last_pymnt_amnt", "collections_12_mths_ex_med", "policy_code", 
    "application_type", "acc_now_delinq", "tot_coll_amt", "tot_cur_bal", 
    "open_acc_6m", "open_act_il", "open_il_12m", "open_il_24m", 
    "mths_since_rcnt_il", "total_bal_il", "il_util", "open_rv_12m", 
    "open_rv_24m", "max_bal_bc", "all_util", "total_rev_hi_lim", "inq_fi", 
    "total_cu_tl", "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", 
    "bc_open_to_buy", "bc_util", "chargeoff_within_12_mths", "delinq_amnt", 
    "mo_sin_old_il_acct", "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", 
    "mo_sin_rcnt_tl", "mort_acc", "mths_since_recent_bc", 
    "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl", 
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0", "num_sats", 
    "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m", 
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", 
    "pub_rec_bankruptcies", "tax_liens", "tot_hi_cred_lim", 
    "total_bal_ex_mort", "total_bc_limit", "total_il_high_credit_limit", 
    "hardship_flag", "debt_settlement_flag",
    "loan_status"
]

target = "loan_status"

In [12]:
# Load the data
df1 = pd.read_csv(Path('../Generator/LoanStats_2019Q1.csv.zip'), skiprows=1)[:-2]
df2 = pd.read_csv(Path('../Generator/LoanStats_2019Q2.csv.zip'), skiprows=1)[:-2]
df3 = pd.read_csv(Path('../Generator/LoanStats_2019Q3.csv.zip'), skiprows=1)[:-2]
df4 = pd.read_csv(Path('../Generator/LoanStats_2019Q4.csv.zip'), skiprows=1)[:-2]

df = pd.concat([df1, df2, df3, df4]).loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)


low_risk_rows = df[df[target] == 'low_risk']
high_risk_rows = df[df[target] == 'high_risk']

#df = pd.concat([low_risk_rows, high_risk_rows.sample(n=len(low_risk_rows), replace=True)])
df = pd.concat([low_risk_rows.sample(n=len(high_risk_rows), random_state=42), high_risk_rows])
df = df.reset_index(drop=True)
df = df.rename({target:'target'}, axis="columns")
df

/Users/jim/anaconda3/envs/mlenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (0,123,124,125,128,129,130,133) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/jim/anaconda3/envs/mlenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (0,123,124,125,128,129,130,133,138,139,140) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/jim/anaconda3/envs/mlenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (0,138,139,140) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
0,1500.0,0.1102,49.13,RENT,30000.0,Not Verified,n,23.04,0.0,0.0,...,50.0,0.0,0.0,28949.0,9084.0,12500.0,16449.0,N,N,low_risk
1,40000.0,0.1390,928.66,MORTGAGE,88000.0,Verified,n,27.26,0.0,0.0,...,80.0,0.0,0.0,217972.0,98128.0,34900.0,73822.0,N,N,low_risk
2,16000.0,0.1033,518.76,MORTGAGE,75000.0,Source Verified,n,9.90,0.0,0.0,...,0.0,1.0,0.0,152207.0,26614.0,6700.0,30307.0,N,N,low_risk
3,7000.0,0.0756,217.94,RENT,75000.0,Source Verified,n,14.50,0.0,0.0,...,33.3,0.0,0.0,73083.0,55917.0,17000.0,49583.0,N,N,low_risk
4,35000.0,0.1774,883.83,RENT,135200.0,Source Verified,n,33.98,0.0,0.0,...,71.4,0.0,0.0,209220.0,120609.0,108900.0,79420.0,N,N,low_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9489,10000.0,0.1695,248.26,MORTGAGE,125000.0,Not Verified,n,6.15,0.0,1.0,...,20.0,0.0,0.0,39620.0,18643.0,18600.0,5600.0,N,N,high_risk
9490,25525.0,0.1862,656.81,MORTGAGE,115000.0,Source Verified,n,18.09,0.0,0.0,...,50.0,0.0,0.0,299752.0,61803.0,12300.0,53588.0,N,N,high_risk
9491,15000.0,0.2055,402.02,OWN,250000.0,Source Verified,n,11.28,3.0,3.0,...,50.0,0.0,0.0,706585.0,58760.0,8800.0,93085.0,N,N,high_risk
9492,14400.0,0.1430,494.26,MORTGAGE,75000.0,Not Verified,n,22.50,0.0,0.0,...,50.0,0.0,0.0,305090.0,36244.0,15100.0,53974.0,N,N,high_risk


In [13]:
df.to_csv('2019loans.csv', index=False)

In [15]:
# Load the data
validate_df = pd.read_csv(Path('../Generator/LoanStats_2020Q1.csv.zip'), skiprows=1)[:-2]
validate_df = validate_df.loc[:, columns].copy()

# Drop the null columns where all values are null
validate_df = validate_df.dropna(axis='columns', how='all')

# Drop the null rows
validate_df = validate_df.dropna()

# Remove the `Issued` loan status
issued_mask = validate_df[target] != 'Issued'
validate_df = validate_df.loc[issued_mask]

# convert interest rate to numerical
validate_df['int_rate'] = validate_df['int_rate'].str.replace('%', '')
validate_df['int_rate'] = validate_df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = dict.fromkeys(['Current', 'Fully Paid'], 'low_risk') 
validate_df = validate_df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period', 'Charged Off'], 'high_risk')    
validate_df = validate_df.replace(x)

low_risk_rows = validate_df[validate_df[target] == 'low_risk']
high_risk_rows = validate_df[validate_df[target] == 'high_risk']

validate_df = pd.concat([low_risk_rows.sample(n=len(high_risk_rows), random_state=37), high_risk_rows])
validate_df = validate_df.reset_index(drop=True)
validate_df = validate_df.rename({target:'target'}, axis="columns")
validate_df

/Users/jim/anaconda3/envs/mlenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (0,138,139,140) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
0,15000.0,0.0646,459.47,MORTGAGE,86000.0,Not Verified,n,33.78,0.0,0.0,...,0.0,0.0,0.0,349427.0,76146.0,50400.0,99924.0,N,N,low_risk
1,24000.0,0.1695,595.82,RENT,80000.0,Not Verified,n,26.70,0.0,0.0,...,60.0,0.0,0.0,117783.0,53423.0,50200.0,50983.0,N,N,low_risk
2,35000.0,0.0819,712.87,MORTGAGE,134000.0,Source Verified,n,19.61,0.0,0.0,...,33.3,0.0,0.0,245017.0,92102.0,56000.0,79806.0,N,N,low_risk
3,12000.0,0.1774,432.27,MORTGAGE,65000.0,Not Verified,n,11.27,0.0,0.0,...,33.3,0.0,0.0,202459.0,16012.0,3300.0,3934.0,N,N,low_risk
4,18475.0,0.2880,584.19,RENT,95000.0,Source Verified,n,53.56,0.0,0.0,...,100.0,0.0,0.0,191246.0,122431.0,26700.0,142746.0,N,N,low_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15235,25000.0,0.2565,1002.62,RENT,60000.0,Source Verified,n,22.44,0.0,0.0,...,62.5,0.0,0.0,57163.0,42830.0,16000.0,41163.0,N,N,high_risk
15236,25000.0,0.1862,911.61,MORTGAGE,160000.0,Not Verified,n,10.12,0.0,3.0,...,66.7,1.0,0.0,55191.0,27274.0,5300.0,48191.0,N,N,high_risk
15237,18000.0,0.1033,385.38,MORTGAGE,240000.0,Source Verified,n,4.96,0.0,0.0,...,60.0,0.0,0.0,33248.0,18155.0,21500.0,10548.0,N,N,high_risk
15238,30000.0,0.1171,662.95,RENT,50000.0,Verified,n,30.41,0.0,5.0,...,0.0,0.0,0.0,143144.0,82750.0,36900.0,104244.0,N,N,high_risk


In [17]:
validate_df.to_csv('2020Q1loans.csv', index=False)